In [3]:
# Uncomment the lines below and run the installations
#!pip install torch
#!pip install torchvision
#!pip install opencv-python
#!pip install gradio

In [4]:
# Importa libs
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os